<a href="https://colab.research.google.com/github/parimalakettymuthu/MachineLearning-Projects/blob/main/Multiclass_Classification_assignment5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [216]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [217]:
import sys 
sys.path.append('/content/drive/MyDrive/NLP/custom-functions')

In [218]:
if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  drive.mount('/content/drive')
  !pip install torchtext --upgrade -qq
  base_path = '/content/drive/MyDrive/NLP/assignment5'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [219]:
from pathlib import Path
import pandas as pd
import torch
import torch.nn as nn
from torchtext.vocab import vocab
from collections import Counter
from torch.utils.data import Dataset, DataLoader


In [220]:
data_folder = Path(base_path)
for entries in data_folder.iterdir():
  print(entries.name)

multiclass_hw_cleaned.csv
Multiclass_Classification_assignment5.ipynb


In [221]:
stackExchange1 = pd.read_csv(data_folder/ 'multiclass_hw_cleaned.csv', encoding = 'ISO-8859-1')
print(stackExchange1.shape)
print(stackExchange1.head(2))

(188878, 6)
   Unnamed: 0                                     Title  \
0           0   detail disclosure indicator on UIButton   
1           1  hello world fails to show up in emulator   

                                                Body  \
0  <p>Is there a simple way to place a detail dis...   
1  <p>I followed Hello World tutorial exactly.  E...   

                                        cleaned_text     Tags  \
0  detail disclosure indicator uibutton simple wa...   iphone   
1  hello world fail emulator follow hello world t...  android   

   Tag_Number_final  
0                 8  
1                 4  


In [222]:
stackExchange = pd.DataFrame().assign(cleaned_text=stackExchange1['cleaned_text'], Tag_Number_final=stackExchange1['Tag_Number_final'])
print(stackExchange)

                                             cleaned_text  Tag_Number_final
0       detail disclosure indicator uibutton simple wa...                 8
1       hello world fail emulator follow hello world t...                 4
2       jshint throw possible strict violation line tr...                 3
3       programmatically bound column invisible try da...                 9
4       edittext get focus soft keyboard android home ...                 4
...                                                   ...               ...
188873  reload tab activity tab change reload activity...                 4
188874  learning index loop list contain list inner li...                 7
188875  module pattern | private access pass reference...                 3
188876  listener homebutton android   possible duplica...                 4
188877  php optimize multiple conditional loop multipl...                 2

[188878 rows x 2 columns]


In [223]:
stackExchange.dropna(inplace=True)
print(stackExchange)

                                             cleaned_text  Tag_Number_final
0       detail disclosure indicator uibutton simple wa...                 8
1       hello world fail emulator follow hello world t...                 4
2       jshint throw possible strict violation line tr...                 3
3       programmatically bound column invisible try da...                 9
4       edittext get focus soft keyboard android home ...                 4
...                                                   ...               ...
188873  reload tab activity tab change reload activity...                 4
188874  learning index loop list contain list inner li...                 7
188875  module pattern | private access pass reference...                 3
188876  listener homebutton android   possible duplica...                 4
188877  php optimize multiple conditional loop multipl...                 2

[188874 rows x 2 columns]


In [224]:
from sklearn.model_selection import train_test_split

# Split data into train and test sets (80/20 split)
SExchangeTrain, test_val = train_test_split(stackExchange, test_size=0.2, random_state=42)

# Split train set into train and validation sets (50/50 split)
SExchangeTest, SExchangeVal = train_test_split(test_val, test_size=0.5, random_state=42)

# Print the shapes & percentage of split of the resulting datasets
print("Train set shape: ", SExchangeTrain.shape, "Split %: ", len(SExchangeTrain)/len(stackExchange))
print("Validation set shape: ", SExchangeVal.shape,"Split %: ", len(SExchangeVal)/len(stackExchange))
print("Test set shape: ", SExchangeTest.shape, "Split %: ", len(SExchangeTest)/len(stackExchange))

Train set shape:  (151099, 2) Split %:  0.7999989410930038
Validation set shape:  (18888, 2) Split %:  0.1000031767209886
Test set shape:  (18887, 2) Split %:  0.09999788218600761


In [225]:
X_SExchangeTrain = SExchangeTrain["cleaned_text"]    #Train, Test & Valid dataset's  cleaned_text X variable
X_SExchangeTest = SExchangeTest["cleaned_text"]
X_SExchangeValid = SExchangeVal["cleaned_text"]

Y_SExchangeTrain = SExchangeTrain["Tag_Number_final"]  #Train, Test & Valid dataset's  Tag_Number_final y variable
Y_SExchangeTest = SExchangeTest["Tag_Number_final"]
Y_SExchangeValid = SExchangeVal["Tag_Number_final"]

In [227]:
!pip install wandb -qq
import wandb
import numpy as np

#Custom class

In [228]:
class CustomClass(Dataset):   #Custom class with init, len, & getitem to initialize and indexing
  def __init__(self, X, y):
    self.X = X
    self.y = y 

  def __len__(self):
    return len(self.X)

  def __getitem__(self, idx):
    text = self.X.iloc[idx]
    labels = self.y.iloc[idx]
    tup = (labels, text)
    return tup

In [229]:
SEtrain_dataset = CustomClass(X_SExchangeTrain, Y_SExchangeTrain)   #Applying CustomClass class on train dataset
i=0
for i, (x,y) in enumerate(SEtrain_dataset):
  if(i<=3):
    print(i, x,y)
    i+=1
  else:
    break

0 7 python read ini file python want transform joomla ini language file sql joomla ini file actually miss section example translation rawconfigparser job demand section construct temp file dummy section name      fout = tempfile namedtemporaryfile(delete = true      fin = file(self._infilename r      fout.write("[all]\n      f              fout.write(f      config = configparser rawconfigparser(allow_no_value = true           c config.items("all              self._ini2sql(unicode(c[0]).upper unicode('de unicode(c[1][1:-1   def elegant solution tip pythonic
1 2 php data difference give fatal error date value come database want calculate difference today date database date date come database 2012 06 11 18:20:40   use code value      echo date('y m d h s      echo $ result['dt_pub_date   write code $ val = date('y m d h s ->diff($result['dt_pub_date   get error fatal error member function diff non object   thank
2 4 convert arraylist message convert arraylist message convert msg arraylist

In [230]:
SEtest_dataset = CustomClass(X_SExchangeTest, Y_SExchangeTest)   #Applying CustomClass class on test dataset
i=0; SEtest_labels=[]
for x,y in SEtest_dataset:
  SEtest_labels.append(int(x))
for i, (x,y) in enumerate(SEtest_dataset):
  if(i<=3):
    print(i, x,y)
    i+=1
  else:
    break

0 4 android tab support v4l2 usb host functionality need know name android tablet v4l2 support host functionality suppose plug usb camera tablet functionality thank lot advance
1 2 filename php $ str = /d file shop/7c0ea5b0b96a490a9c78255ddb559943.jpg   filename path thank lot
2 6 c++ ` my_map[i = ` std::map < int int > my_map my_map[0 =   my_map[0 0 1 undefined
3 0 code path return value method public bool userexistsactivedir     try            const int ads_uf_accountdisable = 0x00000002        directoryentry de = new directoryentry        de path = ldap://domainname        directorysearcher objadsearcher = new directorysearcher(de        de authenticationtype = authenticationtypes secure         objadsearcher searchroot = de        objadsearcher filter = samaccountname= + txtusername text +        searchresult result = objadsearcher findone         result tostring =                  int flag = convert toint32(results properties["useraccountcontrol"][0].tostre           //results pro

In [231]:
SEvalid_dataset = CustomClass(X_SExchangeValid, Y_SExchangeValid)   #Applying CustomClass class on valid dataset
i=0
for i, (x,y) in enumerate(SEvalid_dataset):
  if(i<=3):
    print(i, x,y)
    i+=1

0 3 cross domain ajax activexobject ie activexobject object ie submit cross domain request
1 5 wrap script function professional approach right work little training code try code semi fixed page element result   $ document).ready(function      var elementposition = $ header").offset      var elementheight = $ header").outerheight      $ header").before("<div id='placeholder style='display height + elementheight + px'></div >      $ window).scroll(function          checkattached("#header            function checkattached(element          var windowdepth = $ window).scrolltop          windowdepth > =              $ element).addclass("attached              $ placeholder").css("display block                       $ element).removeclass("attache              $ placeholder").css("display                 thing happy solution like little flexible goal independent function allow address page element like attachthis(element   right have trouble scroll event inside function know good solution ove

#Create Vocab

In [232]:
def createVocab(Dataset):    #General createVocab function to generate count after splitting, insert token, and set default index
  counter = Counter()
  for (label, line) in Dataset:
    counter.update(str(line).split())
  my_vocab = vocab(counter, min_freq=5)    #assinged min frequence as 5
  my_vocab.get_stoi()     #getting the index
  
  my_vocab.insert_token('<unk>', 0) # '<unk>' token for representing any unknown word
  my_vocab.set_default_index(0)   # setting the default index to zero  

  return my_vocab

In [233]:
SETrain_vocab = createVocab(SEtrain_dataset)   #Called the createVocab function for train, test & valid dataset
SETest_vocab = createVocab(SEtest_dataset)
SEValid_vocab = createVocab(SEvalid_dataset)

#collate_batch function

In [234]:
# general function to get the indices of words from vocab
def text_pipeline(x, vocab):
    """Converts text to a list of indices using a vocabulary dictionary"""
    return [vocab[token] for token in str(x).split()]

In [235]:
def collate_batch(batch,vocab):    #general function to create labels, texts & offset based on input 
    labels, texts = zip(*batch)
    labels = torch.tensor(labels, dtype=torch.int64)
    #if 
    list_of_list_of_indices = [text_pipeline(text, vocab) for text in texts]
    
    offsets = [0] + [len(i) for i in list_of_list_of_indices]
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    texts = torch.cat([torch.tensor(i, dtype=torch.int64) for i in list_of_list_of_indices])
    return labels, texts, offsets

#DataLoaders

In [236]:
#Created train, test & valid DataLoader based on input parameters and called collate_batch function
torch.manual_seed(0)
batch_size = 256
SEtrain_loader = torch.utils.data.DataLoader(dataset=SEtrain_dataset,
                                           batch_size=batch_size,
                                           shuffle=False,
                                           collate_fn=lambda batch: collate_batch(batch, SETrain_vocab)
                                           )

SEvalid_loader = torch.utils.data.DataLoader(dataset=SEvalid_dataset,
                                           batch_size=batch_size,
                                           shuffle=False,
                                           collate_fn=lambda batch: collate_batch(batch, SEValid_vocab)
                                           )
SEtest_loader = torch.utils.data.DataLoader(dataset=SEtest_dataset,
                                           batch_size=batch_size,
                                           shuffle=False,
                                           collate_fn=lambda batch: collate_batch(batch, SETest_vocab)
                                           )


#Model

In [239]:
import torch.nn as nn
#Defined the MLPCustom class to instantiate the variables, layers and writted forward to apply the layers required using dropouts
class MLPCustom(nn.Module):
    def __init__(self, vocab, embed_dim=300, n_hidden1=200, n_hidden2=100, n_outs=10, dropout1=0.5, dropout2=0.5):
        super().__init__()

        self.embedding = nn.EmbeddingBag(len(vocab),embed_dim)

        self.hidden_layer1 = nn.Linear(embed_dim, n_hidden1)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(p=dropout1)
        self.batchnorm1 = nn.BatchNorm1d(n_hidden1)

        self.hidden_layer2 = nn.Linear(n_hidden1, n_hidden2)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(p=dropout2)
        self.batchnorm2 = nn.BatchNorm1d(n_hidden2)

        self.output_layer = nn.Linear(n_hidden2,n_outs)

    def forward(self, texts, offsets, vocab):
        # input: tensor of shape (batch_size, seq_len)
        # offsets: tensor of shape (batch_size,) containing the starting indices of each sequence in the input tensor

        embedded = self.embedding(texts, offsets)

        h1 = self.hidden_layer1(embedded)
        h1 = self.relu1(h1)
        h1 = self.dropout1(h1)
        h1 = self.batchnorm1(h1)

        h2 = self.hidden_layer2(h1)
        h2 = self.relu2(h2)
        h2 = self.dropout2(h2)
        h2 = self.batchnorm2(h2)

        ypred = self.output_layer(h2)   

        return ypred

In [240]:
# Initialize a new project
wandb.init(name="imdb_shallow_non_seq", project='nlps23_L7')
#20efc82737d336646269a333588441a0c04c4c0d        --API Key
# Initialized number of epochs, learning rate and batch size
learning_rate = 0.2
wandb.log({'learning_rate': learning_rate})
epochs = 20

# device
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

embed_dim=300
n_hidden1=200
n_hidden2=100
n_outs=10
dropout1=0.5
dropout2=0.5

model = MLPCustom(SETrain_vocab, embed_dim, n_hidden1, n_hidden2, n_outs, dropout1, dropout2)

# loss_function
loss_function = nn.CrossEntropyLoss()

# optimizer SGD
optimizer = torch.optim.SGD(model.parameters(), learning_rate)

#Calling the user definition functions

In [241]:
train_loss_history, train_acc_history, valid_loss_history, valid_acc_history = train(SEtrain_loader,
                                                                                     SEvalid_loader,
                                                                                     model,
                                                                                     optimizer,
                                                                                     loss_function,
                                                                                     epochs,
                                                                                     device,SETrain_vocab,SEValid_vocab)
 

Epoch 1/20
Train Loss: 1.0319 | Train Accuracy: 0.04%
Valid Loss: 2.7657 | Valid Accuracy: 0.11%

Epoch 2/20
Train Loss: 0.7933 | Train Accuracy: 0.04%
Valid Loss: 2.8550 | Valid Accuracy: 0.06%

Epoch 3/20
Train Loss: 0.7302 | Train Accuracy: 0.05%
Valid Loss: 2.9253 | Valid Accuracy: 0.04%

Epoch 4/20
Train Loss: 0.6969 | Train Accuracy: 0.03%
Valid Loss: 2.8994 | Valid Accuracy: 0.07%

Epoch 5/20
Train Loss: 0.6667 | Train Accuracy: 0.04%
Valid Loss: 2.8945 | Valid Accuracy: 0.06%

Epoch 6/20
Train Loss: 0.6445 | Train Accuracy: 0.03%
Valid Loss: 2.9177 | Valid Accuracy: 0.07%

Epoch 7/20
Train Loss: 0.6275 | Train Accuracy: 0.04%
Valid Loss: 2.9963 | Valid Accuracy: 0.06%

Epoch 8/20
Train Loss: 0.6107 | Train Accuracy: 0.05%
Valid Loss: 3.0509 | Valid Accuracy: 0.04%

Epoch 9/20
Train Loss: 0.5994 | Train Accuracy: 0.03%
Valid Loss: 2.9164 | Valid Accuracy: 0.07%

Epoch 10/20
Train Loss: 0.5842 | Train Accuracy: 0.05%
Valid Loss: 3.0010 | Valid Accuracy: 0.05%

Epoch 11/20
Train L

#Get Accuracy, Predictions

In [242]:
# Get the prediction and accuracy for the test dataseta
predictions_test, acc_test = get_acc_pred(SEtest_loader, model, device)

In [243]:
# Test Accuracy
print('Test accuracy', acc_test * 100)

Test accuracy tensor(9.3503, device='cuda:0')


In [244]:
wandb.log({'Test_Acc': acc_test})

#Confusion Matrix for Test Data

In [245]:
#Generated Confusion matrix
wandb.log({"conf_mat": wandb.plot.confusion_matrix(
    probs=None,
    y_true=SEtest_labels,
    preds=predictions_test.to('cpu').numpy(),
    class_names=['c#', 'java', 'php', 'javascript','android', 'jquery', 'c++','python', 'iphone', 'asp.net'])})

In [246]:
wandb.finish()

Test_Acc,▁
Train Accuracy,▄▅▇▂▄▁▄▆▃▇▅▅▇▃▄▄▃▅▅█
Train Loss,█▅▄▄▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁
Valid Accuracy,█▄▂▅▄▅▄▂▅▃▃▁▁▁▄▂▅▂▆▃
Valid Loss,▁▂▄▃▃▄▅▆▄▅▇▅██▇█▇▇██
learning_rate,▁
Test_Acc,0.0935
Train Accuracy,0.00051
Train Loss,0.50218
Valid Accuracy,0.00048
Valid Loss,3.1614
